In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import cv2
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# File paths
mat_file_path = "/Users/keshavsaraogi/data/pose/joints.mat"
image_folder = "/Users/keshavsaraogi/data/pose/images/"

In [ ]:
# Load joint data
data = scipy.io.loadmat(mat_file_path)
joints = data["joints"]  # Shape: (3, 14, 2000)

In [ ]:
# Preprocess images
IMG_SIZE = 128  # Resize images to 128x128
num_images = joints.shape[2]

In [ ]:
X = []
y = []

for img_index in range(num_images):
    img_path = os.path.join(image_folder, f"im{img_index+1:04d}.jpg")
    
    # Read image
    img = cv2.imread(img_path)
    if img is None:
        continue  # Skip missing images

    # Resize and normalize image
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize

    # Get keypoints & normalize to [0,1]
    keypoints = joints[:, :, img_index][:2, :]  # Extract (x, y)
    keypoints = keypoints.flatten()  # Convert to (28,) vector

    X.append(img)
    y.append(keypoints)


In [ ]:
# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [ ]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

In [ ]:
# Build Simple CNN Model
def create_simple_cnn():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(28, activation="linear")  # 14 keypoints (x,y) -> 28 values
    ])
    
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


In [ ]:
# Create and Train Model
model = create_simple_cnn()
model.summary()

In [ ]:
# Train
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Save model
model.save("simple_cnn_pose.keras")
print("Training complete. Model saved as 'simple_cnn_pose.keras'.")

## CUSTOM MADE CNN 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_custom_cnn(input_shape=(128, 128, 3), num_keypoints=28):
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        # Flatten layer
        Flatten(),
        
        # Print shape after Flattening
        Dense(1024, activation='relu'),  # Fixing dense input shape mismatch
        Dropout(0.5),
        
        # Output layer (14 keypoints * 2 coordinates each)
        Dense(num_keypoints, activation='linear')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

    return model

# Initialize model
custom_cnn_model = create_custom_cnn()

# Print summary to verify
custom_cnn_model.summary()


In [ ]:
# Initialize model
custom_cnn_model = create_custom_cnn()

# Print summary
custom_cnn_model.summary()

In [ ]:
# Train the custom model
history = custom_cnn_model.fit(
    X_train, y_train, 
    epochs=25,  # Increase epochs for better performance
    batch_size=32, 
    validation_data=(X_test, y_test)
)

# Save the model
custom_cnn_model.save("custom_cnn_pose.h5")
print("Training complete. Model saved as 'custom_cnn_pose.h5'.")

## HEATMAP REGRESSION

In [ ]:
pip install --upgrade typing_extensions

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import albumentations as A
import scipy.io

In [ ]:
IMG_SIZE = 128
HEATMAP_SIZE = 32
NUM_JOINTS = 14
SIGMA = 2

In [ ]:
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(p=0.1)
])

In [ ]:
joints_data = scipy.io.loadmat("/Users/keshavsaraogi/data/pose/joints.mat")
joints = joints_data["joints"]
joints = np.transpose(joints, (2, 1, 0))

In [ ]:
IMAGE_DIR = "/Users/keshavsaraogi/data/pose/images"
image_files = sorted(os.listdir(IMAGE_DIR))
image_paths = [os.path.join(IMAGE_DIR, f) for f in image_files if f.endswith('.jpg')]

In [ ]:
def generate_heatmap(joints, img_size=IMG_SIZE, heatmap_size=HEATMAP_SIZE, sigma=SIGMA):
    """ Generates heatmaps with 32x32 resolution. """
    heatmaps = np.zeros((heatmap_size, heatmap_size, NUM_JOINTS), dtype=np.float32)

    for i, joint in enumerate(joints):
        x, y, _ = joint
        if x < 0 or y < 0:
            continue
        
        x = int((x / img_size) * heatmap_size)
        y = int((y / img_size) * heatmap_size)

        for xi in range(max(0, x - 3 * sigma), min(heatmap_size, x + 3 * sigma)):
            for yi in range(max(0, y - 3 * sigma), min(heatmap_size, y + 3 * sigma)):
                heatmaps[yi, xi, i] = np.exp(-((xi - x) ** 2 + (yi - y) ** 2) / (2 * sigma ** 2))

    return heatmaps

In [ ]:
import cv2
import numpy as np

X_data = []
Y_data = []

for i, img_path in enumerate(image_paths):
    # Load and preprocess image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize pixel values

    # 🔹 Ensure proper format for augmentation
    augmented = augment(image=img)
    img = augmented['image']

    # 🔹 Ensure correct shape for heatmap (32x32x14)
    heatmap = generate_heatmap(joints[i], img_size=IMG_SIZE, heatmap_size=HEATMAP_SIZE, sigma=2)

    # Append processed data
    X_data.append(img)
    Y_data.append(heatmap)

# Convert lists to NumPy arrays
X_data = np.array(X_data, dtype=np.float32)  # Shape: (num_samples, 128, 128, 3)
Y_data = np.array(Y_data, dtype=np.float32)  # Shape: (num_samples, 32, 32, 14)

# Verify dataset dimensions
print(f"X_data shape: {X_data.shape}")
print(f"Y_data shape: {Y_data.shape}")


In [ ]:
split_idx = int(0.8 * len(X_data))
X_train, X_test = X_data[:split_idx], X_data[split_idx:]
Y_train, Y_test = Y_data[:split_idx], Y_data[split_idx:]

In [ ]:
print("Dataset shapes:")
print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("X_test:", X_test.shape, "Y_test:", Y_test.shape)

In [ ]:
def build_heatmap_model():
    inputs = layers.Input(shape=(128, 128, 3))

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.Conv2DTranspose(256, (3, 3), strides=2, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(NUM_JOINTS, (1, 1), activation='sigmoid', padding='same')(x)
    model = models.Model(inputs, x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    
    return model

In [ ]:
model = build_heatmap_model()
model.summary()

history = model.fit(
    X_train, Y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_test, Y_test)
)

In [ ]:
# ✅ Save the Model
model.save("heatmap_pose_model.keras")
print("✅ Training complete. Model saved as 'heatmap_pose_model.keras'.")


In [ ]:
# ✅ Plot Training Loss & MAE
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.title("Loss Curve")

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel("Epochs")
plt.ylabel("Mean Absolute Error")
plt.legend()
plt.title("MAE Curve")

plt.show()